# Shipping and Conflict

This notebook explores the relationship between trade and conflict in the Red Sea region. The focus is on the Beb El-Mandeb strait and the conflict in Yemen. 

In summary, there was very minor correlation found between fatalities reported by ACLED and shipping movement. The greatest correlation (coeff: 0.1) was between weekly trends in fatalities within 150km of the ports and shipping movement. 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../../data/portwatch/chokepoints_data_02_26_2024.csv")
bab_el_mandeb = df[
    (df["portname"] == "Bab el-Mandeb Strait") & (df["date"] > "2022-12-26")
]

In [3]:
bab_el_mandeb["date"] = pd.to_datetime(bab_el_mandeb["date"])

C:\Users\sahit\AppData\Local\Temp\ipykernel_16840\3150530413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bab_el_mandeb["date"] = pd.to_datetime(bab_el_mandeb["date"])


In [4]:
bab_el_mandeb = (
    bab_el_mandeb.groupby(pd.Grouper(key="date", freq="W"))["n_total"]
    .sum()
    .reset_index()
)  # merge(conflict_by_country, on = 'date')

In [5]:
conflict_by_country = pd.read_csv(
    "../../data/conflict/conflict_by_country_2023-01-01_2024-04-08.csv"
)
conflict_by_port = pd.read_csv(
    "../../data/conflict/conflict_by_port_2023-01-01_2024-04-08.csv"
)

In [6]:
acled = pd.read_csv("../../data/conflict/acled_raw_2023-01-01_2024-04-08.csv")
acled.drop(columns="Unnamed: 0", inplace=True)
red_sea = acled[acled["location"] == "South Red Sea"]
red_sea["event_date"] = red_sea["event_date"].apply(lambda x: pd.to_datetime(x))
red_sea = (
    red_sea.groupby([pd.Grouper(key="event_date", freq="D")])["fatalities"]
    .agg(["sum", "count"])
    .reset_index()
)
red_sea.sort_values(by="event_date", inplace=True)

C:\Users\sahit\AppData\Local\Temp\ipykernel_16840\4202878630.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  red_sea['event_date'] = red_sea['event_date'].apply(lambda x:pd.to_datetime(x))


In [7]:
red_sea["cum_count"] = red_sea["count"].cumsum()

In [8]:
conflict_by_country["event_date"] = pd.to_datetime(conflict_by_country["event_date"])
conflict_by_country.rename(columns={"event_date": "date"}, inplace=True)

In [9]:
conflict_by_port["event_date"] = pd.to_datetime(conflict_by_port["event_date"])
conflict_by_port.rename(columns={"event_date": "date"}, inplace=True)

In [10]:
conflict_by_country.drop(columns="Unnamed: 0", inplace=True)
conflict_by_port.drop(columns="Unnamed: 0", inplace=True)

In [11]:
al_ahmadi = (
    conflict_by_port[conflict_by_port["portname"] == "Al Ahmadi"]
    .groupby(["portname", "date"])[["nrEvents", "nrFatalities"]]
    .sum()
    .reset_index()
)

In [12]:
red_sea.rename(columns={"event_date": "date"}, inplace=True)

In [13]:
data = (
    conflict_by_country[conflict_by_country["country"] == "Yemen"]
    .merge(bab_el_mandeb, on="date")
    .merge(al_ahmadi, on="date")
    .merge(red_sea, on="date")
)
data.rename(
    columns={
        "nrEvents_x": "countryEvents",
        "nrEvents_y": "portEvents",
        "nrFatalities_x": "countryFatalities",
        "nrFatalities_y": "portFatalities",
        "cum_count": "Cumulative Attacks on the Red Sea",
    },
    inplace=True,
)

In [14]:
from datetime import datetime

conflict_date = datetime(2023, 10, 7)
crisis_date = datetime(2023, 11, 17)

In [18]:
data.sort_values(by="date", inplace=True)

In [32]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Legend, Span, LinearAxis, Range1d, Label

# Call this to display plots inline in a Jupyter Notebook
output_notebook()

data["date"] = pd.to_datetime(data["date"])


# Create a figure
p = figure(
    x_axis_type="datetime",
    plot_width=1000,
    plot_height=400,
    title="Comparing Weekly Trends in Conflict and Shipping",
)
p.title.text_font_size = "16pt"

# Plot multiple lines
p1 = p.line(data["date"], data["n_total"], line_width=2, color="#B07AA1")
p.extra_y_ranges = {"y2": Range1d(start=0, end=40)}
p.add_layout(LinearAxis(y_range_name="y2", axis_label="y2"), "right")
# p2 = p.line(data["date"], data["countryEvents"], line_width=2, color="#FF9DA7")
p3 = p.line(
    data["date"],
    data["Cumulative Attacks on the Red Sea"],
    line_width=2,
    color="#9C755F",
    y_range_name="y2",
)

legend_items = [
    ("Number of Vessels", [p1]),
    # ("Number of events in Yemen", [p2]),
    ("Cumulative Number of Attacks", [p3]),
]

# Create the legend manually
legend = Legend(items=legend_items, location="center")

# Add the legend to the plot, but ensure it's outside the main figure
p.add_layout(legend, "right")

# Customize the plot
p.yaxis.axis_label = "Number of Vessels in Bal El-Mandab Strait"
p.right[0].axis_label = "Cumulative Number of Attacks in the Red Sea"
p.legend.location = "top_right"
p.yaxis.axis_label_text_font_style = "normal"

p.add_layout(
    Span(
        location=crisis_date,
        dimension="height",
        line_color="#7C7C7C",
        line_width=2,
        line_dash=(4, 4),
    )
)
p.add_layout(
    Span(
        location=conflict_date,
        dimension="height",
        line_color="#7C7C7C",
        line_width=2,
        line_dash=(4, 4),
    )
)

crisis_label = Label(
    y=250,  # Adjust as needed
    x=crisis_date,
    y_units="screen",
    text="Red Sea Crisis",
    render_mode="css",
    background_fill_color="white",
    background_fill_alpha=0.7,
    text_font_size="9pt",
)

conflict_label = Label(
    y=230,  # Adjust as needed
    x=conflict_date,
    y_units="screen",
    text="Middle East Conflict",
    render_mode="css",
    background_fill_color="white",
    background_fill_alpha=0.7,
    text_font_size="9pt",
)

p.add_layout(crisis_label)
p.add_layout(conflict_label)

# Show the plot
show(p)

Loading BokehJS ...

There is a -0.94 correlation between number of vessels in Bal El-Mandab Strait and the cumulative Attacks in the Red Sea Region. Every additional attack in the Red Sea results in a significant reduction of shipping vessels. 

In [24]:
import numpy as np

np.corrcoef(data["n_total"], data["Cumulative Attacks on the Red Sea"])[0, 1]

-0.9742881744662646

In [13]:
acled = pd.read_csv("../../data/conflict/acled_raw_2023-01-01_2024-02-27.csv")
acled["event_date"] = pd.to_datetime(acled["event_date"])
acled = acled.groupby(["country", "event_date"])["fatalities"].sum().reset_index()

In [14]:
acled.rename(
    columns={"fatalities": "countryFatalities", "event_date": "ymd"}, inplace=True
)

In [15]:
df["ymd"] = pd.to_datetime(df["ymd"])